In [ ]:
import pandas as pd 
import numpy as np
import scipy.stats as ss
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
# set some styling defaults for matplotlib
plt.style.use("seaborn-talk")
mpl.rcParams["figure.dpi"] = 90  # change this to set apparent figure size
mpl.rcParams["figure.figsize"] = (7, 3)
mpl.rcParams["figure.frameon"] = False

# set decimal precision to 3 dec. places
%precision 3

In [ ]:
#%pip install pfilter
#%pip install ipycanvas
# uncomment and run the above if you don't have pfilter and/or ipycanvas installed

In [ ]:
from pfilter import ParticleFilter, gaussian_noise, squared_error, independent_sample
from scipy.stats import norm, gamma, uniform 
from ipycanvas import Canvas, hold_canvas
from ipywidgets import Output, Button, Label


# Example 1: probabilistic filters

## Outcomes
You will understand:
* How probabilistic filtering works
* How to implement basic probabilistic filters
* How filtering can be used to extract hidden states
* How to integrate a particle filter into an interactive system

    


## Goal
* Estimate a hidden state in a continuous process from a stream of observations.
* Be able to deal with missing or noisy observations.
* Be able to project forward into the future as needed.
* Be able to quantify uncertainty and the expected value of possible states.

## Task
We'll build a simple "swipe" gesture recogniser. This is intended to recognise a swipe-left or swipe-right movement. Obviously, this is a relatively simple problem to solve, but we'll see how to properly represent uncertainty when approaching it from a Bayesian perspective.

<img src="imgs/swipe.png" width="50%">

## Process
* We build a **filter**, which in this context means a process that combines observations that occur sequentially.
* Note: it's not really the same as a filter in the sense of signal processing, though the concept of processing sequential signals is the same.
    
* The data generating process is assumed to have some temporal coherence; predictions of the future depend on past states.
* Typically, we make a *Markov assumption*: that the current unobserved state encodes everything we know about the next state.
    
### Predictor-corrector

The concept is simple: we first build a model that just predicts what we think might be going on -- a pure simulator. Then we can take observations and use them to "filter out" predictions that are unrealistic given those observations. This is formulated as a Bayesian belief update.

* Prior at time t + evidence at time t -> posterior at time t

$$P(X_{t+1}|Y_t, X_t) \propto P(Y_t | X_{t+1}) P(X_{t+1})$$
$$P(X_{t+1}|Y_t, X_t) \propto P(Y_t | X_{t+1}) P(X_{t})$$

* (we don't typically care about normalising this distribution, we just want to track *relative* likelihoods of hypotheses)

<img src="imgs/stochastic.png" width="50%">

## Package: `pfilter`

We'll use the `pfilter` package for this example. This implements a simple interface to a particle filter. *Caveat: I wrote `pfilter`, so my definition of simple may not be everyone's!*

A particle filter just represents the current posterior distribution as a collection of samples (definite estimates) and updates them given evidence observed, and a forward model of what evidence *would be expected* for any given sample. We can include dynamics, which specify how the unknown state is believed to be changing over time.

This is a sequential Monte Carlo: we push forward a block of samples through some expected dynamics, compare them to an observation, and reproduce those samples that best approximate the current observation: predict-correct.

## Setup

### Latent variables

We need define the variables we want to infer; we'll form distributions over these. We'll stick to a very simple model, which has three variables:

* `direction`: + or - for left or right swipe
* `phase`: the proportion through the swipe, as a number from 0->1
* `phase_rate`: the current rate of movement through the swipe, as a number from 0->1

From this, we'll compute a distribution over whether or not a gesture is completed, which we can use to trigger actions. We'll do this by computing the how much evidence there is that the user is at the end of a swipe, with a consistent direction.

### Priors
We need to set prior distributions on these variables. We'll assume we can equally likely be going left or right, and that swipes 
start at the beginning (phase close to 0) and have some variable possible rates.

In [ ]:
columns = ["direction", "phase", "phase_rate"]

# some basic guesses for how these variables might start out
# note: direction is actually -1, 0 or 1, but I've used a continuous
# distribution just to make things a bit easier
prior_fn = independent_sample([uniform(loc=-1, scale=1).rvs, 
                                uniform(loc=0.0, scale=0.2).rvs, 
                                norm(loc=0,scale=0.02).rvs])
                               
prior_fn(10)


### Dynamic model
Now we need a model of what would happen in the future *if we had no information about what the user was doing* -- that is, a pure forward simulator. We'll assume that a gesture that is active will continue in the direction it was going, at the same phase rate.

In [ ]:
def dynamics(x):
    # force direction to be -1, 0, 1
    direction = np.sign(np.where(np.abs(x[:,0])<0.5, 0, x[:,0]))
    # phase += direction * phase_rate
    x[:,1] = x[:,1] + direction * x[:,2] 
    return x

In [ ]:
from threading import Event, Thread, Lock


class FilterCanvas:
    def __init__(self, pf, observing=False):        
        self.out = Output()
        self.pf = pf
        self.canvas = Canvas(width=500, height=500)                
        self.mouse_x = 0.0
        
        self.canvas.on_mouse_move(self.handle_mouse_move)
        self.observing = observing        
        self.world_lock = Lock()
        self.stop = Button(icon="stop")
        self.label = Label("")
        self.stop.on_click(self.stop_loop)
        self.stopping = Event()
        self.stopping.clear()

    def stop_loop(self, event):
        self.stopping.set()        
        
    def start(self):
        Thread(target=self.simulate).start()
        
    def handle_mouse_move(self, x, y):        
        if self.stopping.is_set() or not self.observing:
            return 
        self.canvas.fill_style = "red"
        self.canvas.fill_rect(x,y,4)                        
        self.mouse_x = (x - self.canvas.width/2.0) / (self.canvas.width/2.0)        
        
       
    def draw_particles(self, particles, y):
        self.canvas.fill_style = "blue"                
        x = particles[:,1] * np.sign(particles[:,0]) * self.canvas.width/ 4 + self.canvas.width/2                
        self.canvas.fill_rects(x, y, 4)
        
                
    def simulate(self):
        y_pos = 0
        for i in range(20_000):            
            if self.stopping.is_set():
                break
            with hold_canvas():
                particles = self.pf.original_particles
                if not self.observing:
                    y = np.full(len(particles), y_pos) + y_pos
                    self.draw_particles(particles, y)
                    y_pos += 2
                else:
                    y = np.random.normal(self.canvas.height/2, 10, len(particles))
                    self.canvas.clear()                
                    self.draw_particles(particles, y)                
                if self.observing and self.mouse_x is not None:
                    pf.update(np.array([self.mouse_x]))
                else:
                    pf.update() 
                self.mouse_x = None
                time.sleep(0.02)
          
    def draw(self):
        display(self.label)
        display(self.stop)
        display(self.out)
        display(self.canvas)
        

In [ ]:
pf = ParticleFilter(prior_fn=prior_fn, n_particles=100, dynamics_fn=dynamics)        
c = FilterCanvas(pf)
c.draw()
c.start()

### Diffusion
The dynamics we implemented are *deterministic*; they assume that, for example, an initial hypothesis that a gesture is swiping left at 10px per second will continue to behave that way in the future. Given that we know our estimation is uncertain, this is a very strong assumption. A way to ameliorate this is to introduce some **diffusion**: stochastic (random) dynamics, that cause the distribution over latent states to "spread out" over time. This is trivial to implement: we just add a bit of noise to each of our particles at each time step:

In [ ]:
def noise(x):
    return gaussian_noise(x, sigmas=[0.00, 0.01, 0.0001])

If we look at the simulation now, we can see that the particles "wander" rather than following straight line paths.

In [ ]:
pf = ParticleFilter(prior_fn=prior_fn, n_particles=100, dynamics_fn=dynamics, noise_fn=noise)        
c = FilterCanvas(pf)
c.draw()
c.start()

### Observation

So far, we have a forward model: a data generating process that behaves as we expect the hidden state to behave. We need to implement the *corrector* step. To do this, we introduce observations, and specify a function that tells us: given a hidden (latent) state, how likely is this observation? We do this by simply comparing each hypothesised observation against the real one, and weighting the result according to how close they are.

We'll assume all we can see is the mouse position at a given instant.

In [ ]:
def observe(x):
    # return what we *expect* to observe, given the hypotheses we have
    direction = np.sign(np.where(np.abs(x[:,0])<0.5, 0, x[:,0]))
    return np.where(direction==0, np.random.uniform(-1,1,direction.shape), x[:, 1] * direction)

def weight(x, y):
    return squared_error(x, y, sigma=0.005)

In [ ]:
pf = ParticleFilter(prior_fn=prior_fn, n_particles=200, dynamics_fn=dynamics, noise_fn=noise, observe_fn=observe,
                   weight_fn=weight, resample_proportion=0.01)        
c = FilterCanvas(pf, observing=True)
c.draw()
c.start()

## Applying

## Reflection
### What did we gain?
* We were able to track gestures even with not-very-reliable mouse input.
* We maintained uncertainty, and could choose to actuate gestures actions when we were quantifiably sure the intention was there.
* We could display uncertainty in estimation to the user.
* We could also predict the future: this could be useful for display or to reduce apparent latency.
### What was difficult?
* The filter can be tricky to tune, particularly in weighting the hypothesised sensor values against the real ones
* A sample based approach is easy to understand, but isn't the most efficient, and the quality of interaction depends on the sample number.
* Any time we are dealing with uncertainty in the loop we have to find ways of reflecting that to the user -- doing so in effectively can be challenging.

### What else could we do?
* We could obviously extend this to much more complex gestures, or other input devices.
* We could use the uncertainty more intelligently in the interaction
* We could take better advantage of the fact we don't have to be locked to real-time